In [58]:
import gensim.downloader

In [356]:
import nltk
import spacy
from nltk.corpus import stopwords
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity

In [297]:
import numpy as np

In [60]:
model = gensim.downloader.load('word2vec-ruscorpora-300')

In [61]:
with open ('na_zap_fronte.txt', 'r', encoding='windows-1251') as book1:
    zap_front = book1.read()

In [86]:
nlp = spacy.load('ru_core_news_sm')
zap_front_tok = nlp(zap_front)

In [345]:
custom_stop_word = ['это', 'нам', 'x', 'нас', 'ещё', "наш", "который", 'всё',"свой", 'весь']
stop_words = set(stopwords.words('russian'))
stop_words.update(custom_stop_word)

def filtered_book(book_tok):
    
    words = [token.lemma_ for token in book_tok if token.is_alpha and not token.is_stop]
    filtered_lemmas = [token for token in words if token not in stop_words]
    
    return filtered_lemmas

def common_words(filtered_list):

    common_words = Counter(filtered_list)
    common_list = common_words.most_common(100)
    common_list = [item[0] for item in common_list]
    common_list_text = ' '.join(common_list)
    common_tokens = nlp(common_list_text)
    word_tag_list =[]
    for word in common_tokens:
        word_tag = f"{word}_{word.pos_}"
        word_tag_list.append(word_tag)
    return word_tag_list

In [346]:
zap_front_filtered = filtered_book(zap_front_tok)

In [347]:
zap_common_words = common_words(zap_front_filtered)

In [348]:
vec =[]
for word in zap_common_words:
    try:
        vec.append(model.get_vector(word))
    except KeyError as e:
        zap_common_words.remove(word)

In [354]:
similar_pairs = []
for i in range(len(vec)):
    vector1 = vec[i]
    for j in range(i+1, len(vec)):
        vector2 = vec[j]
        cos_similarity = cosine_similarity(vector1.reshape(1, -1), vector2.reshape(1, -1))
        norm_a = np.linalg.norm(vector1)
        norm_b = np.linalg.norm(vector2)
        if cos_similarity > 0.8 and norm_a-norm_b < 0.1: 
            similar_pairs.append((model.most_similar(positive=[vector1], topn=1)[0][0], model.most_similar(positive=[vector2], topn=1)[0][0]))

In [355]:
similar_pairs

[('утро_NOUN', 'вечер_NOUN')]

С помощью косинусного сходства можно определить сонаправлнность векторов (если кс=1->угол между векторами=0, вектора параллельны, если кс=0->укол между векторами=90, вектора перпендикулярны), а с помощью разности норм-сравнить по длине. <br> 
Пара слов среди топ 100 слов в книге На западном фронте без перемен, где косинусное сходство ближе всего к 1, а разность норм векторов минимальная - это пара "утро-ночь"

Если изменить порог для косинусного сходства > 0.7, то появляются следующие пары слов

In [353]:
similar_pairs

[('говорить_VERB', 'сказать_VERB'),
 ('идти_VERB', 'пойти_VERB'),
 ('делать_VERB', 'сделать_VERB'),
 ('час_NOUN', 'утро_NOUN'),
 ('ночь_NOUN', 'вечер_NOUN'),
 ('утро_NOUN', 'вечер_NOUN')]

Здесь вектора слов уже "менее сонаправлены", однако по длине схожи 